# Week 5 18/10/21

## Charge Disproportion Calculator

In [1]:
from pymatgen import Composition

In [2]:
ironOxides = ["FeO", "Fe3O4", "LuFe2O4", "Fe2O3", "BiAlO3", "BiInO3"]

def OxidationStateCalc(formula):
    """Returns a pymatgen list-like object of elements with their respective oxidation states (OS) from a given chemical
    formula."""
    chemical = Composition(formula) #converting formula to pymatgen Composition object
    oxStates = list(chemical.add_charges_from_oxi_state_guesses())
    oxStates = [str(element) for element in oxStates]
    return oxStates #returns the number of elements, each with a charge assigned
                                                         #(multiple instances of an element with different charges will be
                                                         #shown if the OS isn't 'normal' - this is the basis of this program)

#testing
for material in ironOxides:
    print(f"{material}: {list(OxidationStateCalc(material))}")

FeO: ['Fe2+', 'O2-']
Fe3O4: ['Fe2+', 'Fe3+', 'O2-']
LuFe2O4: ['Lu3+', 'Fe2+', 'Fe3+', 'O2-']
Fe2O3: ['Fe3+', 'O2-']
BiAlO3: ['Bi3+', 'Al3+', 'O2-']
BiInO3: ['Bi3+', 'In3+', 'O2-']


In [3]:
def SiteCentredCO(material):
    """Returns a material that is likely to undergo charge disproportionation (CD) (as well as the element that is likely
    to be undergoing CD) from a given formula."""
    oxStates = OxidationStateCalc(material)
    oxStates = [element[:-2] for element in oxStates] #removes the charge from each element, e.g. Fe2+ and Fe3+ -> Fe and Fe
    
    print(oxStates)###############
    elements = list(set(oxStates)) #done to remove duplicates - this is done to avoid analysing an element for CD more than once
    for element in elements:
        print(element)#################
        print(f"Elements: {elements}")##############
        instances = elements.count(element)
        if(instances>1):
            print(element)############
            return material, element
        else:
            print(element)##############
            return None, None #have to write 'None, None' since it's still expected that this function will return two values

siteCOmaterials = {}
for material in ironOxides:
    COmaterial, CDelement = SiteCentredCO(material)
    if(COmaterial != None): #i.e. only try to add a material to siteCOmaterials if SiteCentredCO() returned a material
        siteCOmaterials[COmaterial] = CDelement
print(siteCOmaterials)

['Fe', 'O']
Fe
Elements: ['Fe', 'O']
Fe
['Fe', 'Fe', 'O']
Fe
Elements: ['Fe', 'O']
Fe
['Lu', 'Fe', 'Fe', 'O']
Fe
Elements: ['Fe', 'Lu', 'O']
Fe
['Fe', 'O']
Fe
Elements: ['Fe', 'O']
Fe
['Bi', 'Al', 'O']
Bi
Elements: ['Bi', 'Al', 'O']
Bi
['Bi', 'In', 'O']
Bi
Elements: ['Bi', 'O', 'In']
Bi
{}


### Problem with the above:

In the if/else section of the SiteCenteredCO() function, else returns - returns breaks the for loop and ends the function definition (so, currently, only the first element in the element list is being considered) - need 'else' to simply continue the loop and examine the next element, but if no elements are found to undergo CD, then nothing would be returned, making calling the function difficult, since two items are expected to be returned (see above).

Potential things to look into for SiteCentredCO():
1. How to call the function and its return values.
2. The for loop in the function, and finding an alternative to making else return.

### Version 2 of SiteCentredCO using try/except

In [4]:
def SiteCentredCO(material):
    """Returns a material that is likely to undergo charge disproportionation (CD) (as well as the element that is likely
    to be undergoing CD) from a given formula."""
    oxStates = OxidationStateCalc(material)
    print(f"Elements and their OS: {oxStates}")
    oxStates = [element[:-2] for element in oxStates] #removes the charge from each element, e.g. Fe2+ and Fe3+ -> Fe and Fe
    
    print(f"\noxStates: {oxStates}")###############
    elements = list(set(oxStates)) #done to remove duplicates - this is done to avoid analysing an element for CD more than once
    for element in elements:
        print(f"Elements: {elements}")##############
        print(f"Testing element: {element}")#################
        instances = oxStates.count(element) #originally, this was "elements.count(element)", which was using the non-duplicate
                                            #list. Things why the try below always failed, since no CO materials could ever be
                                            #found
        print(f"Instances of {element}: {instances}")
        if(instances>1):
            print(f"CD element: {element}")############
            return material, element

siteCOmaterials = {}
for material in ironOxides:
    print(f"\n\nMaterial: {material}")
    try:
        print("We're in the start of try")
        COmaterial, CDelement = SiteCentredCO(material)
        print("We're in the middle of try")
        siteCOmaterials[COmaterial] = CDelement
        print("We're in the end of try")
        
    except TypeError:
        print(f"TypeError has occurred - cannot unpack non-iterable NoneType object. Material was {material}")

print(f"\n\nsiteCOmaterials: {siteCOmaterials}")



Material: FeO
We're in the start of try
Elements and their OS: ['Fe2+', 'O2-']

oxStates: ['Fe', 'O']
Elements: ['Fe', 'O']
Testing element: Fe
Instances of Fe: 1
Elements: ['Fe', 'O']
Testing element: O
Instances of O: 1
TypeError has occurred - cannot unpack non-iterable NoneType object. Material was FeO


Material: Fe3O4
We're in the start of try
Elements and their OS: ['Fe2+', 'Fe3+', 'O2-']

oxStates: ['Fe', 'Fe', 'O']
Elements: ['Fe', 'O']
Testing element: Fe
Instances of Fe: 2
CD element: Fe
We're in the middle of try
We're in the end of try


Material: LuFe2O4
We're in the start of try
Elements and their OS: ['Lu3+', 'Fe2+', 'Fe3+', 'O2-']

oxStates: ['Lu', 'Fe', 'Fe', 'O']
Elements: ['Fe', 'Lu', 'O']
Testing element: Fe
Instances of Fe: 2
CD element: Fe
We're in the middle of try
We're in the end of try


Material: Fe2O3
We're in the start of try
Elements and their OS: ['Fe3+', 'O2-']

oxStates: ['Fe', 'O']
Elements: ['Fe', 'O']
Testing element: Fe
Instances of Fe: 1
Element

### Success!

SiteCentredCO finally works, along with the piece of code below it in the above cell.

Next, the above cell needs to be combined into one function, and then comes interacting with the Materials Project database.

# Week 6 25/10/21

## MAPI

### Testing

In [1]:
from pymatgen import MPRester

In [2]:
APIkey = input("Please input your API key: ")
with MPRester(APIkey) as mpr:
    #results = mpr.query()
    criteria = {"elements": {"$all": ["Fe", "O"]}}
    properties = ["pretty_formula", "spacegroup.symbol"]
    results = mpr.query(criteria, properties)
    print(results)
    resultsClean = [list(results[results.index(i)].values()) for i in results]
    #Regarding the above line - i refers to an element in the results list, so results.index(i) means 'return the index
    #from the results list using the element i', which is then being used to acquire the values of each element (where
    #each element in the list is a dictionary), since the values give me the actual results - in this case, the formula
    #and space group symbol. That value then needs to be converted to a list, otherwise the type would be "dict_values".
    print(resultsClean)


Please input your API key:  1TmQ7TKwpVLOiv8Z


[{'pretty_formula': 'Mg14AlFeO16', 'spacegroup.symbol': 'Pmmm'}, {'pretty_formula': 'Mg30FeSnO32', 'spacegroup.symbol': 'P4/mmm'}, {'pretty_formula': 'Mg30VFeO32', 'spacegroup.symbol': 'P4/mmm'}, {'pretty_formula': 'Mg30CdFeO32', 'spacegroup.symbol': 'P4/mmm'}, {'pretty_formula': 'CaMg30FeO32', 'spacegroup.symbol': 'P4/mmm'}, {'pretty_formula': 'Mg30AlFeO32', 'spacegroup.symbol': 'P4/mmm'}, {'pretty_formula': 'La7Sm(Fe2O5)4', 'spacegroup.symbol': 'P1'}, {'pretty_formula': 'Sr4Ca4Mn5Fe3O20', 'spacegroup.symbol': 'P1'}, {'pretty_formula': 'Sr2Ca6Fe3(CoO4)5', 'spacegroup.symbol': 'P1'}, {'pretty_formula': 'Sr6Ca2Fe7CoO24', 'spacegroup.symbol': 'Cmm2'}, {'pretty_formula': 'Sr3Ca5Mn3(FeO4)5', 'spacegroup.symbol': 'P1'}, {'pretty_formula': 'La3SmCr2(FeO6)2', 'spacegroup.symbol': 'Pmm2'}, {'pretty_formula': 'Sr6Ca2Fe3Co5O24', 'spacegroup.symbol': 'P2'}, {'pretty_formula': 'Sm2Fe2Se2O3', 'spacegroup.symbol': 'I4/mmm'}, {'pretty_formula': 'Ca2Fe2O5', 'spacegroup.symbol': 'Pmn2_1'}, {'pretty_for

In [7]:
import pymatgen
print(dir(pymatgen))

['ArrayWithUnit', 'Composition', 'DummySpecie', 'Element', 'FloatWithUnit', 'IMolecule', 'IStructure', 'Lattice', 'MPRester', 'MSONable', 'Molecule', 'MontyDecoder', 'MontyEncoder', 'Orbital', 'PeriodicSite', 'SETTINGS', 'SETTINGS_FILE', 'Site', 'Specie', 'Spin', 'Structure', 'SymmOp', 'Unit', '__author__', '__builtins__', '__cached__', '__doc__', '__email__', '__file__', '__loader__', '__maintainer__', '__maintainer_email__', '__name__', '__package__', '__path__', '__spec__', '__version__', '__warningregistry__', '_load_pmg_settings', 'analysis', 'core', 'electronic_structure', 'entries', 'ext', 'fnmatch', 'get_structure_from_mp', 'io', 'loadfn', 'optimization', 'os', 'symmetry', 'util', 'vis', 'warnings', 'yaml']


In [8]:
from pymatgen import Element
print(dir(Element))

['Ac', 'Ag', 'Al', 'Am', 'Ar', 'As', 'At', 'Au', 'B', 'Ba', 'Be', 'Bh', 'Bi', 'Bk', 'Br', 'C', 'Ca', 'Cd', 'Ce', 'Cf', 'Cl', 'Cm', 'Cn', 'Co', 'Cr', 'Cs', 'Cu', 'Db', 'Ds', 'Dy', 'Er', 'Es', 'Eu', 'F', 'Fe', 'Fl', 'Fm', 'Fr', 'Ga', 'Gd', 'Ge', 'H', 'He', 'Hf', 'Hg', 'Ho', 'Hs', 'I', 'In', 'Ir', 'K', 'Kr', 'La', 'Li', 'Lr', 'Lu', 'Lv', 'Mc', 'Md', 'Mg', 'Mn', 'Mo', 'Mt', 'N', 'Na', 'Nb', 'Nd', 'Ne', 'Nh', 'Ni', 'No', 'Np', 'O', 'Og', 'Os', 'P', 'Pa', 'Pb', 'Pd', 'Pm', 'Po', 'Pr', 'Pt', 'Pu', 'Ra', 'Rb', 'Re', 'Rf', 'Rg', 'Rh', 'Rn', 'Ru', 'S', 'Sb', 'Sc', 'Se', 'Sg', 'Si', 'Sm', 'Sn', 'Sr', 'Ta', 'Tb', 'Tc', 'Te', 'Th', 'Ti', 'Tl', 'Tm', 'Ts', 'U', 'V', 'W', 'Xe', 'Y', 'Yb', 'Zn', 'Zr', '__class__', '__doc__', '__members__', '__module__']


In [9]:
print(Element.__module__)

pymatgen.core.periodic_table


In [10]:
import pymatgen.core.periodic_table as perTable

print(dir(perTable))

['Callable', 'Counter', 'DummySpecie', 'Element', 'Enum', 'FloatWithUnit', 'Length', 'MSONable', 'Mass', 'Optional', 'Path', 'SUPPORTED_UNIT_NAMES', 'Specie', 'Unit', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_pt_data', '_pt_row_sizes', 'combinations', 'f', 'formula_double_format', 'get_el_sp', 'json', 'np', 'open', 'product', 're', 'warnings']


In [11]:
print(perTable.__builtins__)

{'__name__': 'builtins', '__doc__': "Built-in functions, exceptions, and other objects.\n\nNoteworthy: None is the `nil' object; Ellipsis represents `...' in slices.", '__package__': '', '__loader__': <class '_frozen_importlib.BuiltinImporter'>, '__spec__': ModuleSpec(name='builtins', loader=<class '_frozen_importlib.BuiltinImporter'>), '__build_class__': <built-in function __build_class__>, '__import__': <built-in function __import__>, 'abs': <built-in function abs>, 'all': <built-in function all>, 'any': <built-in function any>, 'ascii': <built-in function ascii>, 'bin': <built-in function bin>, 'breakpoint': <built-in function breakpoint>, 'callable': <built-in function callable>, 'chr': <built-in function chr>, 'compile': <built-in function compile>, 'delattr': <built-in function delattr>, 'dir': <built-in function dir>, 'divmod': <built-in function divmod>, 'eval': <built-in function eval>, 'exec': <built-in function exec>, 'format': <built-in function format>, 'getattr': <built-i

Current goal:

Inside the periodic_table documentation for pymatgen, there is a class called "ElementBase", which is the parent class for "Element". "ElementBase" contains a function called "from_Z", which will give me an element's symbol from their atomic number. The plan is to create a list of numbers from 1-the end of the periodic table, which, using list comprehension, I can then convert into another list of all the element symbols, which I can then use for the criteria of my mpr.query to get a list of all the materials in the Materials Project database.

This approach also allows me to exclude certain elements, simply by 'popping' their atomic numbers from the list of atomic numbers prior to converting them to symbols.

### Try 2

In [12]:
from pymatgen.core.periodic_table import Element

In [13]:
Element.from_Z(2)

Element He

This took way too long, but we got there in the end.

In [14]:
import numpy as np

noOfElements = 118 #as of 2021
atomicNos = np.arange(1, noOfElements+1) #the function stops just before the given value (default step = 1)
symbolsTypeElement = [Element.from_Z(z) for z in atomicNos]
symbols = [str(symbol) for symbol in symbolsTypeElement]
print(symbols)

['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt', 'Ds', 'Rg', 'Cn', 'Nh', 'Fl', 'Mc', 'Lv', 'Ts', 'Og']


A list of all the elements has been acquired.

Now, time for a slight detour...

### Extra function - HyphenChain (may potentially be useful for chemsys?)

In [15]:
def HyphenChain(someList):
    
    growingString = None
    for i in range(len(someList)):
        if(i==0):
            growingString = f"{someList[i]}-"
        elif(i < len(someList)-1):
            growingString += f"{someList[i]}-"
        else:
            growingString += someList[i]
    
    return growingString

print(HyphenChain(symbols))

H-He-Li-Be-B-C-N-O-F-Ne-Na-Mg-Al-Si-P-S-Cl-Ar-K-Ca-Sc-Ti-V-Cr-Mn-Fe-Co-Ni-Cu-Zn-Ga-Ge-As-Se-Br-Kr-Rb-Sr-Y-Zr-Nb-Mo-Tc-Ru-Rh-Pd-Ag-Cd-In-Sn-Sb-Te-I-Xe-Cs-Ba-La-Ce-Pr-Nd-Pm-Sm-Eu-Gd-Tb-Dy-Ho-Er-Tm-Yb-Lu-Hf-Ta-W-Re-Os-Ir-Pt-Au-Hg-Tl-Pb-Bi-Po-At-Rn-Fr-Ra-Ac-Th-Pa-U-Np-Pu-Am-Cm-Bk-Cf-Es-Fm-Md-No-Lr-Rf-Db-Sg-Bh-Hs-Mt-Ds-Rg-Cn-Nh-Fl-Mc-Lv-Ts-Og


### Back to what I was doing before

In [16]:
def ListOfTheElements(elementsExcluded=None):
    #try/except used to avoid having errors in this Jupyter Notebook
    try:
        noOfElements = 118 #as of 2021
        atomicNos = np.arange(1, noOfElements+1) #the function stops just before the given value (default step = 1)

        if(elementsExcluded != None):
            atomicNos = [list(atomicNos).remove(z) for z in elementsExcluded] #incorrect syntax for this specific attempt at list
                                                                              #comprehension

        symbolsTypeElement = [Element.from_Z(z) for z in atomicNos]
        symbols = [str(symbol) for symbol in symbolsTypeElement]

        return symbols
    
    except ValueError:
        print("ValueError: No element with this atomic number None")
    
print(ListOfTheElements()) # works
print()
print(ListOfTheElements([1, 2])) # doesn't work like anticipated

['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt', 'Ds', 'Rg', 'Cn', 'Nh', 'Fl', 'Mc', 'Lv', 'Ts', 'Og']

ValueError: No element with this atomic number None
None


In [17]:
aListOfNumbers = list(np.arange(10))
numbersToRemove = [1, 2, 6]
aListMissingNumbers = [i for i in aListOfNumbers if i not in numbersToRemove]
print(aListOfNumbers)
print(aListMissingNumbers)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[0, 3, 4, 5, 7, 8, 9]


Now with this better understanding of list comprehension:

### Try 3

In [18]:
def ListOfTheElements(elementsExcluded=None):
    noOfElements = 118 #as of 2021
    atomicNos = np.arange(1, noOfElements+1) #the function stops just before the given value (default step = 1)
    
    if(elementsExcluded != None):
        atomicNos = [z for z in atomicNos if z not in elementsExcluded]
    
    symbolsTypeElement = [Element.from_Z(z) for z in atomicNos]
    symbols = [str(symbol) for symbol in symbolsTypeElement]
    
    return symbols
    
print(ListOfTheElements()) # works
print()
print(f"{ListOfTheElements([1, 2])} H and He removed") #now also works

['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt', 'Ds', 'Rg', 'Cn', 'Nh', 'Fl', 'Mc', 'Lv', 'Ts', 'Og']

['Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'C

#### Removing radioactive elements

In [19]:
radElements = [43, 61]+list(np.arange(84, 118+1)) #atomic nos.: 43, 61, 84-118. https://www.epa.gov/radiation/radioactive-decay
radElementSymbols = [str(Element.from_Z(radE)) for radE in radElements] #converting the atomic nos. into Element-type
                                                                        #variables, and then converting them into strings
print(f"Radioactive elements: {radElementSymbols}")

Radioactive elements: ['Tc', 'Pm', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt', 'Ds', 'Rg', 'Cn', 'Nh', 'Fl', 'Mc', 'Lv', 'Ts', 'Og']


#### Testing if the zip() function can be used to tranpose a 2D array:

In [20]:
TwoDArray = [["a", "b", "c"], ["d", "e", "f"]]
print(TwoDArray)
TwoDArrayT = list(zip(*TwoDArray))
#more info on using asterisks in python:
# https://stackoverflow.com/questions/9754453/in-python-what-type-of-object-does-ziplist1-list2-return
# https://www.technovelty.org/python/on-asterisks-in-python.html
print(TwoDArrayT)
TwoDArrayTT = list(zip(*TwoDArrayT))
print(TwoDArrayTT)
#interestingly, the elements inside the list become tuples after using list(zip(*arg)) - this shouldn't affect what I want to do

[['a', 'b', 'c'], ['d', 'e', 'f']]
[('a', 'd'), ('b', 'e'), ('c', 'f')]
[('a', 'b', 'c'), ('d', 'e', 'f')]


### Putting it all together: Preliminary search program for non-radioactive materials with charge disproportionation

In [ ]:
from pymatgen import Composition
from pymatgen import MPRester
from pymatgen.core.periodic_table import Element
import numpy as np
import matplotlib.pyplot as plt
from json_tricks import dumps, loads #the json module doesn't support non-standard types (such as the output from MAPI),
                                     #but json_tricks does

ironOxides = ["FeO", "Fe3O4", "LuFe2O4", "Fe2O3", "BiAlO3", "BiInO3"]

def OxidationStateCalc(formula):
    """Returns a pymatgen list-like object of elements with their respective oxidation states (OS) from a given chemical
    formula."""
    chemical = Composition(formula) #converting formula to pymatgen Composition object
    oxStates = list(chemical.add_charges_from_oxi_state_guesses())
    oxStates = [str(element) for element in oxStates]
    return oxStates #returns the number of elements, each with a charge assigned
                                                         #(multiple instances of an element with different charges will be
                                                         #shown if the OS isn't 'normal' - this is the basis of this program)

def SiteCentredCO(material):
    """Returns a material that is likely to undergo charge disproportionation (CD) (as well as the element that is likely
    to be undergoing CD) from a given formula."""
    oxStates = OxidationStateCalc(material)
    print(f"Elements and their OS: {oxStates}")
    oxStates = [element[:-2] for element in oxStates] #removes the charge from each element, e.g. Fe2+ and Fe3+ -> Fe and Fe
    
    print(f"\noxStates: {oxStates}")###############
    elements = list(set(oxStates)) #done to remove duplicates - this is done to avoid analysing an element for CD more than once
    for element in elements:
        print(f"Elements: {elements}")##############
        print(f"Testing element: {element}")#################
        instances = oxStates.count(element) #originally, this was "elements.count(element)", which was using the non-duplicate
                                            #list. Things why the try below always failed, since no CO materials could ever be
                                            #found
        print(f"Instances of {element}: {instances}")
        if(instances>1):
            print(f"CD element: {element}")############
            return material, element

def CheckForCD(listOfMaterials):
    """Takes in a list and returns a dictionary of materials with they key as the material that seems to have undergone charge
    disproportionation (CD), and the value as the element undergoing CD."""
    
    siteCOmaterials = {}
    for material in listOfMaterials:
        print(f"\n\nMaterial: {material}")
        try:
            COmaterial, CDelement = SiteCentredCO(material)
            siteCOmaterials[COmaterial] = CDelement

        except TypeError:
            pass
            #TypeError has occurred - cannot unpack non-iterable NoneType object. Material was {material}"

    return siteCOmaterials       


def ListOfTheElements(elementsExcluded=None):
    noOfElements = 118 #as of 2021
    atomicNos = np.arange(1, noOfElements+1) #the function stops just before the given value (default step = 1)
    
    if(elementsExcluded != None):
        atomicNos = [z for z in atomicNos if z not in elementsExcluded]
    
    symbolsTypeElement = [Element.from_Z(z) for z in atomicNos]
    symbols = [str(symbol) for symbol in symbolsTypeElement]
    
    return symbols

def CleanUpResults(results):
    """Will take a MAPI output, take only the values of each dictionary element, and make the elements into lists of
    those values - it makes the results easier to deal with."""
    resultsClean = [list(results[results.index(i)].values()) for i in results]
    #Regarding the above line - i refers to an element in the results list, so results.index(i) means 'return the index
    #from the results list using the element i', which is then being used to acquire the values of each element (where
    #each element in the list is a dictionary), since the values give me the actual results - in this case, the formula
    #and space group symbol. That value then needs to be converted to a list, otherwise the type would be "dict_values".
    
    resultsCleanTranspose = list(zip(*resultsClean)) #separates properties into separate lists, e.g.
    #one element will be a list of "pretty_formula"s and the other would be "spacegroup.symbol"s.
    
    return resultsCleanTranspose



#Now it's time for the search
radElements = [43, 61]+list(np.arange(84, 118+1)) #atomic nos.: 43, 61, 84-118. https://www.epa.gov/radiation/radioactive-decay
radElementSymbols = [str(Element.from_Z(radE)) for radE in radElements] #converting the atomic nos. into Element-type
                                                                        #variables, and then converting them into strings
print(f"Radioactive elements: {radElementSymbols}")

nonRadElements = ListOfTheElements(radElements) #list of elements to search with
print(nonRadElements)
APIkey = input("Please input your API key: ")





results = None #done so that results exists outside the scope of the with block
with MPRester(APIkey) as mpr:
    criteria = {"elements": {"$in": nonRadElements}} #want to find materials that contain any of the listed elements, hence $in
    properties = ["pretty_formula", "spacegroup.symbol"]
    results = mpr.query(criteria, properties)
    
    #^ currently outputting 0, since my criteria states that all of the elements I listed should be in a compound
    #- impossible - need to look at MongoDB operator syntax again (fixed - wrote '$all' instead of '$in' - see above)
    #resultsCD = CheckForCD(results) - currently won't work since 'results' isn't just a list.
    #First, I need to make a list of 'pretty_formula's, and then I can use the CD function
    #- maybe create a function that does this? Issue is that I'll have different queries every time, so there's not much point
    #in making a general function.

    with open("NonRadSearch.txt", "w") as f: #this is the thing I'm working on now
        f.write(dumps())
    
#new resultsCD using 'CleanUpResults':
resultsClean = CleanUpResults(results)
formulas = resultsClean.pop(0) #removing and saving the first element from resultsClean
spaceGroupSymbols = resultsClean.pop(0) #will use later for finding most common space groups for CD FE candidates
# ^ regarding the last line, the pop index has to be 0 since the previous pop removed the item which originally had index 0
resultsCD = CheckForCD(formulas)

#will want to include space group plotting thingy since this process takes so long. Actually, may want to write results to a
#file to do further (and faster) analysis later (I don't want to lok through the entire database everytime I want to analyse
#my results).

Radioactive elements: ['Tc', 'Pm', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Es', 'Fm', 'Md', 'No', 'Lr', 'Rf', 'Db', 'Sg', 'Bh', 'Hs', 'Mt', 'Ds', 'Rg', 'Cn', 'Nh', 'Fl', 'Mc', 'Lv', 'Ts', 'Og']
['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi']


Since Jupyer notebook automatically prints results, without my asking, the print statements to gauge how much culling my CD function has done will be below:

In [ ]:
print(f"No. of results before checking for CD: {len(results)}")
print(f"No, of results after checking for CD: {len(resultsCD)}")
print(f"Percentage of materials that are FE candidates based on charge disproportion: {len(resultsCD)/len(results)}")